<a href="https://colab.research.google.com/github/lantrungseo-oss/fastformer-for-rec/blob/main/Microsoft_News_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/MyDrive/data

behaviors.tsv.zip	  news.tsv.zip
entity_embedding.vec.zip  relation_embedding.vec


In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import pairwise_distances
import copy
from IPython.display import clear_output
import warnings
from re import sub
import plotly
import plotly.express as px
import matplotlib.pyplot as plt

plotly.offline.init_notebook_mode (connected = True)

import random
warnings.filterwarnings("ignore")
from tqdm import tqdm
import plotly.io as pio
pio.renderers.default = 'colab'

In [ ]:
!pip install faiss
!pip install -U sentence-transformers


     |████████████████████████████████| 4.7 MB 10.9 MB/s 
     |████████████████████████████████| 79 kB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 15.7 MB/s 
     |████████████████████████████████| 1.2 MB 9.8 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 41.8 MB/s 
     |████████████████████████████████| 596 kB 41.0 MB/s 
     |████████████████████████████████| 6.5 MB 45.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=05d0746f539dbbc1d8559f5d43ae1df786fce2cf305fb26bf5b85728464653ff
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install faiss-cpu
!pip install transformers
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 8.6 MB 14.8 MB/s 
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1 MB 1.8 kB/s 
     |████████████████████████████████| 8.8 MB 43.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.2.0+cu92 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.2.0+cu92 which is incompatible.
sentence-transformers 2.

In [ ]:
!unzip /content/gdrive/MyDrive/data/news.tsv.zip

Archive:  /content/gdrive/MyDrive/data/news.tsv.zip
  inflating: news.tsv                


In [ ]:
!ls

gdrive	news.tsv  sample_data


In [ ]:
data=pd.read_csv('news.tsv',header=None,sep='\t')


In [ ]:
data.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7], dtype='int64')

In [ ]:
data.sample(10)

,0,1,2,3,4,5,6,7
31536,N1937,travel,traveltips,The worst tourist trap in every state,"These attractions reel vacationers in daily, b...",https://assets.msn.com/labs/mind/BBSnHNQ.html,[],[]
28630,N51893,travel,travelarticle,Author's book describes 100 things to do in R....,On the first day writer Robert Curley set foot...,https://assets.msn.com/labs/mind/AAIGehJ.html,"[{""Label"": ""Rhode Island"", ""Type"": ""G"", ""Wikid...","[{""Label"": ""Rhode Island"", ""Type"": ""G"", ""Wikid..."
37183,N53572,music,musicnews,"Taylor Swift Rep Hits Back at Big Machine, Cla...",Taylor Swift's team responds to Big Machine's ...,https://assets.msn.com/labs/mind/BBWP294.html,"[{""Label"": ""Big Machine Records"", ""Type"": ""O"",...","[{""Label"": ""Big Machine Records"", ""Type"": ""O"",..."
25333,N28587,sports,basketball_nba,"14-year-old girl disappears from school, deput...",The Pasco County Sheriff's Office is searching...,https://assets.msn.com/labs/mind/AAIPzbw.html,[],"[{""Label"": ""Pasco County Sheriff's Office"", ""T..."
23299,N7297,finance,markets,U.K. to Destroy Commemorative 50p Coins in Bre...,Thousands of 50-pence coins minted to commemor...,https://assets.msn.com/labs/mind/AAJur9K.html,"[{""Label"": ""Brexit"", ""Type"": ""U"", ""WikidataId""...","[{""Label"": ""Boris Johnson"", ""Type"": ""P"", ""Wiki..."
10891,N56098,news,newsus,"Apartments for rent in Mesa: What will $1,200 ...",Curious just how far your dollar goes in Mesa ...,https://assets.msn.com/labs/mind/AAJgHRG.html,"[{""Label"": ""Mesa, Arizona"", ""Type"": ""G"", ""Wiki...","[{""Label"": ""Mesa, Arizona"", ""Type"": ""G"", ""Wiki..."
24295,N41219,foodanddrink,tipsandtricks,5 foods you should always buy frozen instead o...,"Plus, meal ideas from a dietitian to ensure th...",https://assets.msn.com/labs/mind/AAIG1Ls.html,[],[]
43726,N54114,news,newscrime,Three White Plains men charged in major drug ring,Three White Plains men were charged in a major...,https://assets.msn.com/labs/mind/BBWI0Cf.html,"[{""Label"": ""White Plains, New York"", ""Type"": ""...","[{""Label"": ""White Plains, New York"", ""Type"": ""..."
41150,N27238,news,newsscienceandtechnology,He Thought His Phone Was Secure; Then He Lost ...,Security researchers agree that for most peopl...,https://assets.msn.com/labs/mind/BBWtiRD.html,[],[]
19527,N54029,news,newsworld,Firefighter hit by car on interstate credits c...,NaN,https://assets.msn.com/labs/mind/AAJ42Ov.html,[],[]


In [ ]:
data.columns=['News ID',
"Category",
"SubCategory",
"Title",
"Abstract",
"URL",
"Title Entities",
"Abstract Entities "]

In [ ]:
data=data.iloc[:,:5]


In [ ]:
data.sample(10)

,News ID,Category,SubCategory,Title,Abstract
15114,N28830,lifestyle,lifestylebuzz,Tree Toppled On Greensboro Home In Storm Narro...,A Greensboro couple is breathing a sigh of rel...
39282,N45939,travel,travelnews,Plans for luxury Mandarin Oriental hotel annou...,It's already a prime location and in the next ...
40752,N18793,news,newsus,Off-duty officers were the first responders to...,The first responders reported to the scene at ...
9829,N20959,health,health-news,High school sports with highest concussion rat...,The rates of football practice concussions and...
43045,N19685,news,newspolitics,Brett Kavanaugh calls Ruth Bader Ginsburg 'ins...,In his first speech as a Supreme Court justice...
24548,N16288,weather,weathertopstories,Another National Retailer Closing 8 CT Locatio...,Ex-fire official arrested/ Town Attorney mourn...
36555,N49876,sports,football_ncaa,Florida Gators football: Gators out to prevent...,"Heading into this season, the Florida Gators h..."
49659,N16166,news,newsus,San Francisco's new DA pledges not to prosecut...,San Francisco's newly elected district attorne...
27340,N51850,sports,basketball_ncaa,Jeff Goodman's Top 102 College Basketball Play...,Find out which hoopers made the cut for Jeff G...
4776,N3120,lifestyle,lifestylebuzz,Photos: Elevator at Harvard Square MBTA statio...,The project had a price tag of $3.8 million.


In [ ]:
def generate_mapping_index(dataframe):

  index_map = {}

  for index, row in dataframe.iterrows():
    index_map[index] = {
        "News ID": row['News ID'],
        "Title" : row['Title'],
        "Abstract" : row['Abstract'],
        "Category" : row['Category'],
        "SubCategory" : row['SubCategory']
    }

  return index_map

index_map = generate_mapping_index(data)

In [ ]:
c=data[['Category','SubCategory']].value_counts()

In [ ]:
index=[]
for i in c.index:
    index.append(np.array(i))
index=np.array(index)

In [ ]:
df=pd.DataFrame(columns=['Category','Sub Category','Values'])
df['Category']=index[:,0]
df['Sub Category']=index[:,1]
df['Values']=c.values

In [ ]:

px.bar(data_frame=df,x='Category',y='Values',color='Sub Category')

In [ ]:
t=[]
for i in data['Title']:
    t.append(len(i))
px.histogram(t,color=data['Category'])

In [ ]:
data.dropna(inplace=True)

In [ ]:
print('the number of articles before processing :',len(data))
data.drop_duplicates(subset=['Title'],inplace=True)
print('The number of articles after processing :',len(data))

the number of articles before processing : 48616
The number of articles after processing : 47799


In [ ]:
print('the number of articles before processing :',len(data))
data=data[data['Title'].apply((lambda x: len(x.split())>=4))]
print('The number of articles after processing :',len(data))

the number of articles before processing : 47799
The number of articles after processing : 47672


In [ ]:
import transformers
import torch

model_name  = 'bert-base-uncased'

model = transformers.BertModel.from_pretrained(model_name)

tokenizer = transformers.BertTokenizer.from_pretrained(model_name, do_lower_case=True)

device = torch.device('cuda:0')

model.to(device)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
import torch


def embedding_fn(model, text) :

  with torch.no_grad():
    #generate tokens :
    tokens = tokenizer.encode(text)
    #expand dims :
    batch_tokens = np.expand_dims(tokens, axis = 0)
    batch_tokens = torch.tensor(batch_tokens).cuda()
    #print(type(batch_tokens))
    #generate embedding and return hidden_state :
    return model(batch_tokens)[0].cpu()

def embedding_fn_cpu(model, text) :

  with torch.no_grad():
    #generate tokens :
    tokens = tokenizer.encode(text, max_length = 512)
    #expand dims :
    batch_tokens = np.expand_dims(tokens, axis = 0)
    batch_tokens = torch.tensor(batch_tokens)
    #print(type(batch_tokens))
    #generate embedding and return hidden_state :
    return model(batch_tokens)[0]

def compute_mean(embedding):

  if not isinstance(embedding, torch.Tensor):
    print('Embedding must be a torch.Tensor')
    return

  return embedding.mean(1)


def compute_cosine_measure(x1, x2):

  #given two points in vector space, measure cosine distance
  return cosine_similarity(x1, x2)


def compute_distance(x1, x2):
  #replace this with your own measure
  return compute_cosine_measure(x1.detach().numpy(), x2.detach().numpy())


In [ ]:
CHUNK_SIZE_EACH = 44000



def __embedding(text):
  return compute_mean(embedding_fn(model, text))



def compute_bert_embeddings(dataframe_chunk, current_index, end_marker):

  np_chunk = __embedding(dataframe_chunk.loc[current_index * end_marker]['Title']).detach().numpy()
  #np_chunk = np_chunk.reshape(np_chunk.shape[1])

  for idx in range(1, end_marker):

    try:
      embedding = __embedding(dataframe_chunk.loc[(current_index * end_marker) + idx]['Title']).detach().numpy()
      #embedding = embedding.reshape(embedding.shape[1])
      np_chunk = np.append(np_chunk, embedding, axis = 0)
      print('\r {}'.format(np_chunk.shape), end = '')
    except Exception as e:
      print(e)
      np_chunk = np.append(np_chunk, np.zeros(shape = (1, 768)), axis = 0)
      continue

  print(np_chunk.shape)
  np.savez_compressed('title_{}'.format(current_index), a = np_chunk)


def compute_embeddings_and_save(dataframe):

  n_rows = len(dataframe)

  chunk_sizes = n_rows // CHUNK_SIZE_EACH
  remaining = n_rows - chunk_sizes * CHUNK_SIZE_EACH

  for i in range(1):

    compute_bert_embeddings(dataframe[i * CHUNK_SIZE_EACH : (i * CHUNK_SIZE_EACH) + CHUNK_SIZE_EACH ], i, CHUNK_SIZE_EACH)



In [ ]:
#compute_embeddings_and_save(data)

In [ ]:
!ls gdrive

MyDrive


In [ ]:
#!cp title_0.npz gdrive/MyDrive/
!cp gdrive/MyDrive/title_0.npz ./

In [ ]:
embeddings = np.load('title_0.npz')['a']
embeddings.shape

(44000, 768)

In [ ]:
import faiss

n_dimensions = embeddings.shape[1] #Number of dimensions (764)

fastIndex = faiss.IndexFlatL2(n_dimensions) # We will create an index of type FlatL2, there are many kinds of indexes, you can look at it in their repo.
fastIndex.add(embeddings.astype('float32')) # Add the embedding vector to faiss index, it should of dtype 'float32'


In [ ]:
import time
def do_faiss_lookup(fastIndex, query_text, model, top_k):
    n_embeddings = embeddings.shape[0]
    embedding_q = compute_mean(embedding_fn(model, query_text)).detach().numpy()

    #let it be float32
    embedding_q = embedding_q.astype('float32')

    #perform the search
    st = time.time()
    matched_em, matched_indexes = fastIndex.search(embedding_q, top_k) # it returns matched vectors and thier respective indexes, we are interested only in indexes.

    #indexes are already sorted wrt to closest match
    et = time.time()

    return et - st, matched_indexes[0]

In [ ]:
requested_query = 'Anthony interviews Santa'

In [ ]:
time_faiss_cpu, indexes_top_faiss = do_faiss_lookup(fastIndex, requested_query, model, 20)
print('Faiss index lookup time :  ', time_faiss_cpu, ' seconds')

def index_to_title(indexes):

    for i, idx in enumerate(indexes) :
        print('{}. {}'.format(i, index_map[idx]['Title']))

index_to_title(indexes_top_faiss)

Faiss index lookup time :   0.03726840019226074  seconds
0. Anthony interviews Santa
1. Shaking Beef
2. Remembering Elijah Cummings
3. Meet Victor the cameraman eagle
4. The Worst Debate Question
5. Dad Dressed As Superhero Surprises Daughter
6. Celebrity plastic surgery transformations
7. Panel finds Portsmouth dog is vicious
8. Celebrity Halloween costume fails
9. Russell Westbrook takes shot at Patrick Beverley
10. Salmon Recalled In Michigan
11. Salmon Recalled In Michigan
12. The News In Cartoons
13. The News In Cartoons
14. Ford Kills the Flex
15. Winthrop takes on ETSU
16. The Exercise Coach comes to Westchase
17. FRIENDS Moist Maker Sandwich Is Monica Level Perfection
18. Woods to play upgraded Genesis Invitational
19. 2020 NASCAR Mustang Breaks Cover
